# **Computational Drug Discovery - Obtaining Bioactivity Data from ChEMBL**

Machine learning model using bioactivity data from ChEMBL Database.

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database containing curated bioactivity data of more than 2.1 million compounds compiled from more than 81,000 documents, 1.4 million assays and the data spans 14,000 targets and 2,000 cells and 40,000 indications.
[Data as of January 11, 2022; ChEMBL version 29].

## **Installing libraries**

Install ChEMBL web service package to retrieve bioactivity data from the ChEMBL Database.

In [ ]:
! pip install chembl_webresource_client

     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 596 kB 16.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: itsdangerous
    Found existing installation: itsdangerous 1.1.0
    Uninstalling itsdangerous-1.1.0:
      Successfully uninstalled itsdangerous-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires itsdangerous<2.0,>=0.24, but you have itsdangerous 2.0.1 which is incompatible.


## **Importing libraries**

In [ ]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for coronavirus**

In [ ]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('coronavirus')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Coronavirus,Coronavirus,17.0,False,CHEMBL613732,[],ORGANISM,11119
1,[],SARS coronavirus,SARS coronavirus,15.0,False,CHEMBL612575,[],ORGANISM,227859
2,[],Feline coronavirus,Feline coronavirus,15.0,False,CHEMBL612744,[],ORGANISM,12663
3,[],Human coronavirus 229E,Human coronavirus 229E,13.0,False,CHEMBL613837,[],ORGANISM,11137
4,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
5,[],Middle East respiratory syndrome-related coron...,Middle East respiratory syndrome-related coron...,9.0,False,CHEMBL4296578,[],ORGANISM,1335626
6,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.0,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
7,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


### **Select and retrieve bioactivity data for specific entry using entry ID**

We will assign the seventh entry (which corresponds to the target protein, *Replicase polyprotein 1ab*) to the ***selected_target*** variable 

In [ ]:
selected_target = targets.target_chembl_id[6]
selected_target

'CHEMBL5118'

Here, we will retrieve only bioactivity data for *Replicase polyprotein 1ab* (CHEMBL5118) that are reported as IC$_{50}$ values in nM (nanomolar) unit.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(5)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,1988091,[],CHEMBL898907,Inhibition of SARS-CoV 3C-like protease by FRE...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCOC(=O)/C=C/[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@@H...,Outside typical range,Values for this activity type are unusually la...,CHEMBL1137503,Bioorg. Med. Chem. Lett.,2007,None,CHEMBL194398,None,CHEMBL194398,None,False,http://www.openphacts.org/units/Nanomolar,663132,=,1,True,=,None,IC50,nM,None,870000.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,227859,None,None,IC50,uM,UO_0000065,None,870.0
1,None,1988092,[],CHEMBL898907,Inhibition of SARS-CoV 3C-like protease by FRE...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCOC(=O)/C=C/[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,Outside typical range,Values for this activity type are unusually la...,CHEMBL1137503,Bioorg. Med. Chem. Lett.,2007,None,CHEMBL393608,None,CHEMBL393608,None,False,http://www.openphacts.org/units/Nanomolar,663138,=,1,True,=,None,IC50,nM,None,200000.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,227859,None,None,IC50,uM,UO_0000065,None,200.0
2,None,1988093,[],CHEMBL898907,Inhibition of SARS-CoV 3C-like protease by FRE...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCOC(=O)/C=C/[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,Outside typical range,Values for this activity type are unusually la...,CHEMBL1137503,Bioorg. Med. Chem. Lett.,2007,None,CHEMBL238216,None,CHEMBL238216,None,False,http://www.openphacts.org/units/Nanomolar,663137,=,1,True,=,None,IC50,nM,None,300000.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,227859,None,None,IC50,uM,UO_0000065,None,300.0
3,None,1988094,[],CHEMBL898907,Inhibition of SARS-CoV 3C-like protease by FRE...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCOC(=O)/C=C/[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,None,None,CHEMBL1137503,Bioorg. Med. Chem. Lett.,2007,"{'bei': '7.31', 'le': '0.14', 'lle': '3.95', '...",CHEMBL235873,None,CHEMBL235873,4.82,False,http://www.openphacts.org/units/Nanomolar,663136,=,1,True,=,None,IC50,nM,None,15000.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,227859,None,None,IC50,uM,UO_0000065,None,15.0
4,None,1988095,[],CHEMBL898907,Inhibition of SARS-CoV 3C-like protease by FRE...,B,None,None,BAO_0000190,BAO_0000019,assay format,CCOC(=O)/C=C/[C@H](C[C@@H]1CCNC1=O)NC(=O)[C@H]...,None,None,CHEMBL1137503,Bioorg. Med. Chem. Lett.,2007,"{'bei': '7.99', 'le': '0.16', 'lle': '4.33', '...",CHEMBL397154,None,CHEMBL397154,5.00,False,http://www.openphacts.org/units/Nanomolar,663135,=,1,True,=,None,IC50,nM,None,10000.0,CHEMBL5118,SARS coronavirus,Replicase polyprotein 1ab,227859,None,None,IC50,uM,UO_0000065,None,10.0


In [ ]:
df.standard_type.unique()

array(['IC50'], dtype=object)

We will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [ ]:
df.to_csv('bioactivity_data.csv', index=False)

## **Copying files to Google Drive**

Firstly, we need to sync the Google Drive into Colab so that we can have access to our Google Drive from within Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)


MessageError: ignored

Next, we create a **data** folder in our **Colab Notebooks** folder on Google Drive.

In [ ]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data"

total 1995
-rw------- 1 root root 2042499 Jan 11 05:19 bioactivity_data.csv


Let's see the CSV files that we have so far.

In [ ]:
! ls

bioactivity_data.csv  bioactivity_preprocessed_data.csv  gdrive  sample_data


Take a glimpse of the **bioactivity_data.csv** file created.

In [ ]:
! head bioactivity_data.csv

activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
,32264,[],CHEMBL845865,Inhibition of autophosphorylation of human Her (p185erbB) tyrosine kinase expressed in SKOV-3 cells,F,,,BAO_0000190,BAO_0000219,cell-based format,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)c32)[nH]1,,,CHEMBL1134862,Bioorg. Med. Chem. Lett.,2002.0,,CHEMBL68920,,CHEMBL68920,6.52,False,http://www.o

## **Handling missing data**
If any compounds has missing value for the **standard_value** column then it will be removed from the dataset

In [ ]:
df2 = df[df.standard_value.notna()]
df2

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,bao_label,canonical_smiles,data_validity_comment,data_validity_description,document_chembl_id,document_journal,document_year,ligand_efficiency,molecule_chembl_id,molecule_pref_name,parent_molecule_chembl_id,pchembl_value,potential_duplicate,qudt_units,record_id,relation,src_id,standard_flag,standard_relation,standard_text_value,standard_type,standard_units,standard_upper_value,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,32264,[],CHEMBL845865,Inhibition of autophosphorylation of human Her...,F,None,None,BAO_0000190,BAO_0000219,cell-based format,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,None,None,CHEMBL1134862,Bioorg. Med. Chem. Lett.,2002.0,None,CHEMBL68920,None,CHEMBL68920,6.52,False,http://www.openphacts.org/units/Nanomolar,119482,=,1,True,=,None,IC50,nM,None,300.0,CHEMBL1824,Homo sapiens,Receptor protein-tyrosine kinase erbB-2,9606,None,None,IC50,uM,UO_0000065,None,0.3
1,None,32266,[],CHEMBL615491,Inhibition of ligand induced proliferation in ...,F,None,None,BAO_0000190,BAO_0000219,cell-based format,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,None,None,CHEMBL1134862,Bioorg. Med. Chem. Lett.,2002.0,None,CHEMBL68920,None,CHEMBL68920,5.60,False,http://www.openphacts.org/units/Nanomolar,119482,=,1,True,=,None,IC50,nM,None,2500.0,CHEMBL1824,Homo sapiens,Receptor protein-tyrosine kinase erbB-2,9606,None,None,IC50,uM,UO_0000065,None,2.5
2,None,32271,[],CHEMBL683802,Inhibition of autophosphorylation of human Her...,F,None,None,BAO_0000190,BAO_0000219,cell-based format,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,None,None,CHEMBL1134862,Bioorg. Med. Chem. Lett.,2002.0,None,CHEMBL69960,None,CHEMBL69960,6.40,False,http://www.openphacts.org/units/Nanomolar,119494,=,1,True,=,None,IC50,nM,None,400.0,CHEMBL1824,Homo sapiens,Receptor protein-tyrosine kinase erbB-2,9606,None,None,IC50,uM,UO_0000065,None,0.4
3,None,32273,[],CHEMBL615491,Inhibition of ligand induced proliferation in ...,F,None,None,BAO_0000190,BAO_0000219,cell-based format,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,None,None,CHEMBL1134862,Bioorg. Med. Chem. Lett.,2002.0,None,CHEMBL69960,None,CHEMBL69960,5.92,False,http://www.openphacts.org/units/Nanomolar,119494,=,1,True,=,None,IC50,nM,None,1210.0,CHEMBL1824,Homo sapiens,Receptor protein-tyrosine kinase erbB-2,9606,None,None,IC50,uM,UO_0000065,None,1.21
4,None,47937,[],CHEMBL683802,Inhibition of autophosphorylation of human Her...,F,None,None,BAO_0000190,BAO_0000219,cell-based format,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,None,None,CHEMBL1134862,Bioorg. Med. Chem. Lett.,2002.0,None,CHEMBL67057,None,CHEMBL67057,7.00,False,http://www.openphacts.org/units/Nanomolar,119500,=,1,True,=,None,IC50,nM,None,100.0,CHEMBL1824,Homo sapiens,Receptor protein-tyrosine kinase erbB-2,9606,None,None,IC50,uM,UO_0000065,None,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3857,None,20708048,[],CHEMBL4628614,Inhibition of HER2 (unknown origin) using FAM-...,B,None,None,BAO_0000190,BAO_0000357,single protein format,O=C(/C=C/CN1CCCC1)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,None,None,CHEMBL4627322,Bioorg Med Chem Lett,2020.0,"{'bei': '14.63', 'le': '0.28', 'lle': '2.28', ...",CHEMBL4635100,None,CHEMBL4635100,8.40,False,http://www.openphacts.org/units/Nanomolar,3487704,=,1,True,=,None,IC50,nM,None,4.0,CHEMBL1824,Homo sapiens,Receptor protein-tyrosine kinase erbB-2,9606,None,None,IC50,uM,UO_0000065,None,0.004
3858,None,20708049,[],CHEMBL4628614,Inhibition of HER2 (unknown origin) using FAM-...,B,None,None,BAO_0000190,BAO_0000357,single protein format,CCN(CC)C/C=C/C(=O)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,None,None,CHEMBL4627322,Bioorg 

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
IC50 unit represents bioactivity data. Compounds with values of <1000 nM will be considered to be **active** while those >10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [ ]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

### **Iterate the *molecule_chembl_id* to a list**

In [ ]:
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

### **Iterate *canonical_smiles* to a list**

In [ ]:
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

### **Iterate *standard_value* to a list**

In [ ]:
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

### **Combine the 4 lists into a dataframe**

In [ ]:
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [ ]:
df3

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,active,300.0
1,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,intermediate,2500.0
2,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,active,400.0
3,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,intermediate,1210.0
4,CHEMBL67057,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,active,100.0
...,...,...,...,...
2929,CHEMBL4635100,O=C(/C=C/CN1CCCC1)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,active,4.0
2930,CHEMBL4648271,CCN(CC)C/C=C/C(=O)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,active,4.0
2931,CHEMBL554,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...,active,13.0
2932,CHEMBL939,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1,active,281.0


### **Alternative method**

In [ ]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2[selection]
df3

,molecule_chembl_id,canonical_smiles,standard_value
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,300.0
1,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,2500.0
2,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,400.0
3,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,1210.0
4,CHEMBL67057,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,100.0
...,...,...,...
3857,CHEMBL4635100,O=C(/C=C/CN1CCCC1)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,4.0
3858,CHEMBL4648271,CCN(CC)C/C=C/C(=O)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,4.0
3859,CHEMBL554,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...,13.0
3860,CHEMBL939,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1,281.0


In [ ]:
pd.concat([df3,pd.Series(bioactivity_class)], axis=1)

,molecule_chembl_id,canonical_smiles,standard_value,0
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,300.0,active
1,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,2500.0,intermediate
2,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,400.0,active
3,CHEMBL69960,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,1210.0,intermediate
4,CHEMBL67057,Cc1cc(C(=O)N2CCOCC2)[nH]c1/C=C1\C(=O)Nc2ncnc(N...,100.0,active
...,...,...,...,...
3857,CHEMBL4635100,O=C(/C=C/CN1CCCC1)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,4.0,NaN
3858,CHEMBL4648271,CCN(CC)C/C=C/C(=O)N1CCOc2cc3ncnc(Nc4ccc(OCc5cc...,4.0,NaN
3859,CHEMBL554,CS(=O)(=O)CCNCc1ccc(-c2ccc3ncnc(Nc4ccc(OCc5ccc...,13.0,NaN
3860,CHEMBL939,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1,281.0,NaN


Saves dataframe to CSV file

In [ ]:
df3.to_csv('bioactivity_preprocessed_data.csv', index=False)

In [ ]:
! ls -l

total 2236
-rw-r--r-- 1 root root 2042499 Jan 11 05:18 bioactivity_data.csv
-rw-r--r-- 1 root root  233541 Jan 11 05:24 bioactivity_preprocessed_data.csv
drwx------ 5 root root    4096 Jan 11 05:18 gdrive
drwxr-xr-x 1 root root    4096 Jan  7 14:33 sample_data


Let's copy to the Google Drive

In [ ]:
! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

In [ ]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data"

bioactivity_data.csv  bioactivity_preprocessed_data.csv


---